# 4 Local approximations


The volume of a four-bar truss should be minimized under the displacement constraint $\delta \le \delta_0$. There is a force $P>0$ and all bars have identical lengths $l$ and identical Young's moduli $E$. The modifiable structural variables are the cross sectional areas $A_1=A_4$ and $A_2=A_3$. We define $A_0 = Pl / (10\delta_0E)$ and constrain the variables $0.2A_0 \le A_j \le 2.5 A_0$. Then we can use dimensionless design variables $a_j=A_j/A_0 \in [0.2, 2.5]$.


![Four bar truss](figures/four_bar_truss.png)


Credits: Peter W. Christensen and Anders Klarbring. *An Introduction to Structural Optimization.* Springer Netherlands, 2008.

In [4]:
import torch

## Task 1 - Defining the constrained optimization problem

a) Compute the objective function $f(\mathbf{a})$ that should be minimized and define it as Python function that accepts inputs tensors of the shape [..., 2].


> The total volume is 
> $$ V = l A_1 + l A_2 + l A_3 + l A_4 = 2l A_1 + 2l A_2.$$
> Substituting $A_j = a_j A_0$ gives 
> $$ V = 2l A_0 (a_1 + a_2).$$
> The factor in front of the brackets is a positive constant, which we can neglect for the optimization task itself. Hence, we can simplify the function to
> $$f(\mathbf{a}) = a_1 + a_2.$$

In [5]:
def f(a):
    return a[..., 0] + a[..., 1]

b) Compute the constraint function $g(\mathbf{a})$ for $\delta_0=0.1$ and define it as Python function that accepts inputs tensors of the shape [..., 2].

> A free body diagram of the free node gives 
> $$ - \frac{4}{5} P_1 - \frac{3}{5} P_2 + \frac{3}{5}P_3 + \frac{4}{5} P_4 + \frac{3}{5} P = 0 $$
> $$ \frac{3}{5} P_1 + \frac{4}{5} P_2 + \frac{4}{5}P_3 + \frac{3}{5} P_4 + \frac{4}{5} P = 0 $$
> In addition, the kinematics give us the following relation for the elongation of each bar $\Delta u_j$:
> $$ \frac{4}{5} u_1 - \frac{3}{5} u_2 = \Delta u_1 $$
> $$ \frac{3}{5} u_1 - \frac{4}{5} u_2 = \Delta u_2 $$
> $$ -\frac{3}{5} u_1 - \frac{4}{5} u_2 = \Delta u_3 $$
> $$ -\frac{4}{5} u_1 - \frac{3}{5} u_2 = \Delta u_4 $$
> Finally, we also have a elastic relation for each bar: 
> $$P_1 = E A_1 \frac{\Delta u_1}{l}$$
> $$P_2 = E A_2 \frac{\Delta u_2}{l}$$
> $$P_3 = E A_3 \frac{\Delta u_3}{l}$$
> $$P_4 = E A_4 \frac{\Delta u_4}{l}$$


In [6]:
def g(a):
    return (
        8 / (16 * a[..., 0] + 9 * a[..., 1])
        - 4.5 / (9 * a[..., 0] + 16 * a[..., 1])
        - 0.1
    )

c) Summarize the optimization problem statement with all constraints. 

## Task 2 - CONLIN

a) Implement a function named `CONLIN(func, a_k, a)`


In [7]:
def CONLIN(func, a_k, a):
    a_lin = a_k.clone().requires_grad_()
    grads = torch.autograd.grad(func(a_lin).sum(), a_lin)[0]
    res = func(a_k) * torch.ones_like(a[..., 0])
    for i, grad in enumerate(grads):
        if grad < 0:
            gamma = a_k[i] / a[..., i]
        else:
            gamma = 1
        res += grad * gamma * (a[..., i] - a_k[i])
    return res